# Building a Fractional Pseudorandom Generator

**Paper**: Chattopadhyay, Hatami, Lovett, Tal. [Pseudorandom Generators from the Second Fourier Level and Applications to AC0 with Parity Gates](https://doi.org/10.4230/LIPIcs.ITCS.2019.22). *ITCS 2019*.

---

This notebook walks through the construction of a **fractional pseudorandom generator** from the paper. The construction itself has several moving parts, so it helps to see each one happen concretely. At the end, we use the construction as an excuse to poke at Conjecture 3 on small instances.

### What the paper does

A PRG for a function class $\mathcal{F}$ produces samples that no $f \in \mathcal{F}$ can distinguish from truly random inputs. The paper builds this in two stages:

1. A **fractional PRG** -- samples in $[-1,1]^n$ that fool $f$'s multilinear extension, using only $\ell \ll n$ random values.
2. A **polarizing random walk** (Theorem 7) converts the fractional output to Boolean $\{-1,+1\}^n$.

We implement stage 1. The paper's target application is AC0[$\oplus$] circuits, but the construction works for any restriction-closed class with bounded $L_{1,2}$. We test it on degree-$d$ polynomials over $\mathbb{F}_2$, which is the class the paper focuses on.

### Roadmap

1. Background: function classes, Fourier tails, the $L_{1,2}$ condition
2. Test functions: random degree-2 $\mathbb{F}_2$ polynomials
3. Theorem 9: small-variance Gaussians fool bounded-tail functions
4. Dimension reduction via balanced codes
5. The complete fractional PRG
6. Poking at Conjecture 3

In [ ]:
!pip install --upgrade boofun -q

import numpy as np
import matplotlib.pyplot as plt
import boofun as bf
from boofun.analysis.fourier import fourier_level_lp_norm
from boofun.analysis.gf2 import gf2_degree
from scipy.stats import norm
from itertools import combinations

np.random.seed(42)
print(f"boofun version: {bf.__version__}")

## 1. Background

### Function classes

The paper works with a **class** $\mathcal{F}$ of Boolean functions that is **closed under restrictions**: if $f \in \mathcal{F}$ and you fix some variables to constants, the restricted function is still in $\mathcal{F}$. This property is needed because the PRG construction progressively restricts variables.

The class we use: **degree-$d$ polynomials over $\mathbb{F}_2$**, denoted $\text{Poly}_{n,d}$. A member is $f(x) = (-1)^{p(x)}$ where $p : \mathbb{F}_2^n \to \mathbb{F}_2$ is a polynomial of degree $\leq d$. For example, with $d = 2$ on 4 variables:

$$p(x) = x_0 x_1 \oplus x_2 x_3 \oplus x_0 \qquad (\text{degree 2})$$

Fixing $x_0 = 1$ gives $p(x) = x_1 \oplus x_2 x_3 \oplus 1$, still degree $\leq 2$. The class is restriction-closed.

### Fourier tails

Every $f: \{-1,+1\}^n \to \{-1,+1\}$ has a unique multilinear expansion $\tilde{f}(x) = \sum_{S \subseteq [n]} \hat{f}(S) \prod_{i \in S} x_i$. The **level-$k$ Fourier tail** sums the absolute values of coefficients at degree $k$:

$$L_{1,k}(f) = \sum_{|S|=k} |\hat{f}(S)|$$

The paper's main result (Theorem 2): if $\mathcal{F}$ is restriction-closed and $L_{1,2}(\mathcal{F}) \leq t$, then an explicit PRG exists with seed length $\text{poly}(t, \log n, 1/\varepsilon)$. Only the second level matters.

### What is $t$ for our class?

For $\text{Poly}_{n,d}$, the paper proves $L_{1,2}(\text{Poly}_{n,d}) \leq 4 \cdot 2^{6d}$ (exponential in $d$). **Conjecture 3** says the truth is $O(d^2)$ -- polynomial, not exponential. If true, this would give PRGs for AC0[$\oplus$].

In the paper, $t$ is an analytical worst-case bound over the entire class. Here, we compute $L_{1,2}$ exactly for specific instances. The PRG we build would fool any function with $L_{1,2} \leq t$.

## 2. Test Functions: Random Degree-2 $\mathbb{F}_2$ Polynomials

We generate random quadratic polynomials over $\mathbb{F}_2$ on $n = 12$ variables. Each polynomial is a random subset of the $\binom{12}{2} = 66$ quadratic monomials plus the 12 linear monomials.

In [ ]:
n = 12
d = 2


def random_f2_poly(n, d, rng):
    """Generate a random degree-d polynomial over F2 on n variables."""
    monomials = []
    # Include each possible monomial of degree 1..d with probability 1/2
    for deg in range(1, d + 1):
        for combo in combinations(range(n), deg):
            if rng.random() < 0.5:
                monomials.append(set(combo))
    if not monomials:
        # Avoid the constant function
        monomials = [{0, 1}]
    return bf.f2_polynomial(n, monomials)


rng = np.random.default_rng(seed=42)
test_fns = {f"quad_{i}": random_f2_poly(n, d, rng) for i in range(5)}

# Compute Fourier tails
print(f"Random degree-{d} F2 polynomials on n = {n} variables")
print(f"Known bound: L_{{1,2}}(Poly_{{n,{d}}}) <= 4 * 2^(6*{d}) = {4 * 2**(6*d)}")
print(f"Conjecture 3: L_{{1,2}} = O(d^2) = O({d**2})")
print()

header = f"{'Function':<12}{'GF2 deg':<10}" + "".join(
    f"{'L_{1,' + str(k) + '}':<12}" for k in range(5)
)
print(header)
print("-" * 82)

l12_values = {}
for name, func in test_fns.items():
    gf2_d = gf2_degree(func)
    row = f"{name:<12}{gf2_d:<10}"
    for k in range(5):
        val = fourier_level_lp_norm(func, k)
        row += f"{val:<12.4f}"
        if k == 2:
            l12_values[name] = val
    print(row)

t = max(l12_values.values())
print(f"\nEmpirical t = max L_{{1,2}} across our instances: {t:.4f}")
print(f"Known bound for d={d}: {4 * 2**(6*d)}")
print(f"Conjectured bound for d={d}: O({d**2})")

In [ ]:
# Fast batch evaluator: builds all 2^n monomials in O(n * 2^n) per batch
def eval_multilinear_batch(fourier_coeffs, n_vars, X, batch_size=100):
    """Evaluate multilinear extension at rows of X using monomial-building trick."""
    m = X.shape[0]
    size = len(fourier_coeffs)
    results = np.zeros(m)
    for start in range(0, m, batch_size):
        end = min(start + batch_size, m)
        batch = X[start:end]
        b = batch.shape[0]
        # vals[i, S] = product_{j in S} batch[i, j]
        vals = np.zeros((b, size))
        vals[:, 0] = 1.0
        for i in range(n_vars):
            step = 1 << i
            vals[:, step : 2 * step] = vals[:, :step] * batch[:, i : i + 1]
        results[start:end] = vals @ fourier_coeffs
    return results

## 3. Theorem 9: Gaussians Fool Bounded-Tail Functions

The construction relies on this result (restating Raz-Tal):

> **Theorem 9.** Let $Z \in \mathbb{R}^n$ be a zero-mean multivariate Gaussian with $\text{Var}[Z_i] \leq p$ and $|\text{Cov}[Z_i, Z_j]| \leq \delta$ for $i \neq j$. If $L_{1,2}(\mathcal{F}) \leq t$, then for any $f \in \mathcal{F}$:
>
> $$|\mathbb{E}[\tilde{f}(\text{trnc}(Z))] - \tilde{f}(\mathbf{0})| \leq O(\delta \cdot t)$$

where $\text{trnc}$ clips each coordinate to $[-1, 1]$, and $\tilde{f}(\mathbf{0}) = \hat{f}(\emptyset) = \mathbb{E}[f]$.

The error is $O(\delta \cdot t)$ because the level-1 terms vanish (zero mean), and the level-2 terms each pick up a factor of $\delta$ (the covariance). Higher levels contribute negligibly when the variance is small.

To achieve fooling error $\varepsilon$: set $\delta = \varepsilon / t$.

As a baseline, we first try $n$ **independent** Gaussians (trivially $\delta = 0$, but no randomness savings).

In [ ]:
# Parameters from Theorem 9
eps = 0.3
delta = eps / max(t, 1e-6)
p_var = 1.0 / (8 * np.log(n / delta))  # variance per coordinate
sigma = np.sqrt(p_var)

print("Theorem 9 parameters:")
print(f"  n = {n},  t = {t:.4f},  eps = {eps}")
print(f"  delta = eps/t = {delta:.6f}")
print(f"  p (variance) = {p_var:.6f},  sigma = {sigma:.6f}")
print()

# --- Naive baseline: n independent Gaussians ---
n_samples = 500
naive_results = {}
for name, func in test_fns.items():
    fourier = func.fourier()
    E_f = fourier[0]
    Z = np.random.randn(n_samples, n) * sigma
    X = np.clip(Z, -1, 1)
    vals = eval_multilinear_batch(fourier, n, X)
    err = abs(np.mean(vals) - E_f)
    naive_results[name] = {"E_f": E_f, "mean": np.mean(vals), "error": err}
    print(f"  {name}: E[f]={E_f:+.4f}, naive mean={np.mean(vals):+.4f}, error={err:.4f}")

print(f"\nNaive approach: {n} independent random values per sample (no savings).")

## 4. Dimension Reduction via Balanced Codes

Instead of $n$ independent Gaussians, the paper generates $n$ *correlated* values from only $\ell$ independent Gaussians.

**Construction** (Paper, Section 2):
1. Choose codewords $c_1, \ldots, c_n \in \{0,1\}^\ell$ from a $\delta$-balanced code
2. Build $A \in \mathbb{R}^{n \times \ell}$: $\; A_{i,j} = \sqrt{p/\ell} \cdot (-1)^{c_{i,j}}$
3. Sample $Y \sim N(0, I_\ell)$ -- only $\ell$ random values
4. Set $Z = AY$

Each row of $A$ has squared norm $p$, so $\text{Var}[Z_i] = p$ exactly. The covariance between $Z_i$ and $Z_j$ is $p \cdot (1 - 2 d_H(c_i, c_j)/\ell)$, where $d_H$ is the Hamming distance. A good code keeps all pairwise Hamming distances close to $\ell/2$, which makes the covariances small.

The required code length is $\ell = O(\log n / \delta^{2+o(1)})$. At our scale ($n = 12$) the theoretical $\ell$ exceeds $n$ -- the savings only appear for large $n$. We use $\ell = 8$ here to show the mechanics of the construction.

In [ ]:
# Dimension reduction: n -> ell
ell_theory = int(np.ceil((max(t, 1) / eps) ** 2 * np.log2(n) * 10))
ell = 8  # for demonstration

print(f"Dimension reduction: {n} -> {ell}")
print(f"  (Theoretical ell ~ {ell_theory}; at this scale ell > n.)")
print(f"  (The savings appear when n is large and t is moderate.)")
print()

# Balanced codewords: c_i in {0,1}^ell
# The paper uses explicit small-biased spaces (Ta-Shma 2017).
# Here: random codewords, then check properties.
code_rng = np.random.default_rng(seed=7)
codewords = code_rng.integers(0, 2, size=(n, ell))

# Matrix A: A_{i,j} = sqrt(p / ell) * (-1)^{c_{i,j}}
scale = np.sqrt(p_var / ell)
A = scale * (1 - 2 * codewords.astype(float))

# Covariance structure: Cov(Z) = A @ A^T
cov_Z = A @ A.T
diag = np.diag(cov_Z)
off_diag = cov_Z[np.triu_indices(n, k=1)]
effective_delta = np.max(np.abs(off_diag))

print("Covariance structure of Z = AY:")
print(f"  Var[Z_i] = {diag[0]:.6f}  (= p, by construction)")
print(f"  max |Cov[Z_i, Z_j]| = {effective_delta:.6f}  (effective delta)")
print(f"  mean |Cov[Z_i, Z_j]| = {np.mean(np.abs(off_diag)):.6f}")
print(f"\nTheoretical fooling bound: O(delta * t) = {effective_delta * t:.4f}")

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
im0 = axes[0].imshow(A, cmap="RdBu", aspect="auto")
axes[0].set_xlabel(f"Seed dimension ($\ell$ = {ell})")
axes[0].set_ylabel(f"Output dimension (n = {n})")
axes[0].set_title(f"Matrix A  ({n} x {ell})")
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(cov_Z, cmap="RdBu")
axes[1].set_title(f"Covariance  $AA^T$  ({n} x {n})")
plt.colorbar(im1, ax=axes[1])

plt.tight_layout()
plt.show()

## 5. The Complete Fractional PRG

**Discretization** (Paper, Lemma 10-11): each of the $\ell$ continuous Gaussians is approximated by a discrete random variable using $k$ bits.

The full pipeline:

$$\underbrace{s \text{ bits}}_{\text{seed}} \;\xrightarrow{\text{quantize}}\; \underbrace{Y \in \mathbb{R}^\ell}_{\text{approx. Gaussians}} \;\xrightarrow{Z = AY}\; \underbrace{Z \in \mathbb{R}^n}_{\text{correlated}} \;\xrightarrow{\text{trnc}}\; \underbrace{X \in [-1,1]^n}_{\text{output}}$$

We check: $|\mathbb{E}[\tilde{f}(X)] - \mathbb{E}[f]| \leq O(\delta \cdot t)$ for each test function.

In [ ]:
# --- Discretization ---
bits_per_coord = 8
num_levels = 2**bits_per_coord
total_seed_bits = ell * bits_per_coord

# Quantized Gaussian: 256-level approximation of N(0,1)
quantized_gaussian = norm.ppf((np.arange(num_levels) + 0.5) / num_levels)

print(f"Discretization: {bits_per_coord} bits/coord, {num_levels} levels")
print(f"Total seed: {ell} x {bits_per_coord} = {total_seed_bits} bits")
print()

# === Complete Fractional PRG ===
n_trials = 500

print(f"Verification ({n_trials} trials)")
print(f"  Seed: {total_seed_bits} bits -> {ell} Gaussians -> {n} fractional values")
print()

prg_results = {}
for name, func in test_fns.items():
    fourier = func.fourier()
    E_f = fourier[0]

    seeds = np.random.randint(0, num_levels, size=(n_trials, ell))
    Y_batch = quantized_gaussian[seeds]
    Z_batch = Y_batch @ A.T
    X_batch = np.clip(Z_batch, -1, 1)

    vals = eval_multilinear_batch(fourier, n, X_batch)
    prg_mean = np.mean(vals)
    prg_err = abs(prg_mean - E_f)
    theoretical = effective_delta * t

    prg_results[name] = {"E_f": E_f, "mean": prg_mean, "error": prg_err}

    print(f"  {name}:")
    print(f"    E[f] = {E_f:+.4f}")
    print(f"    PRG mean  = {prg_mean:+.4f}  (error = {prg_err:.4f})")
    print(f"    Naive mean = {naive_results[name]['mean']:+.4f}  (error = {naive_results[name]['error']:.4f})")
    print(f"    Theoretical bound ~ {theoretical:.4f}")
    print()

# --- Comparison plot ---
names = list(test_fns.keys())
x_pos = np.arange(len(names))
width = 0.25

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

naive_errs = [naive_results[nm]["error"] for nm in names]
prg_errs = [prg_results[nm]["error"] for nm in names]

axes[0].bar(x_pos - width / 2, naive_errs, width, label=f"Naive ({n} vals)", color="steelblue")
axes[0].bar(x_pos + width / 2, prg_errs, width, label=f"PRG ({ell} vals)", color="orange")
axes[0].axhline(
    y=effective_delta * t,
    color="red",
    linestyle="--",
    label=f"O(delta * t) ~ {effective_delta * t:.4f}",
)
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(names, fontsize=8, rotation=15)
axes[0].set_ylabel("|mean - E[f]|")
axes[0].set_title("Fooling Error")
axes[0].legend(fontsize=7)
axes[0].grid(True, alpha=0.3, axis="y")

# Seed length scaling
ns = [16, 64, 256, 1024, 4096, 16384, 65536]
ell_values = [int(np.ceil((max(t, 1) / eps) ** 2 * np.log2(nn) * 5)) for nn in ns]
axes[1].plot(ns, ns, "k--", label="n (naive)", linewidth=2)
axes[1].plot(ns, ell_values, "o-", color="orange", label=r"$\ell$ (PRG)", linewidth=2)
crossover = next((nn for nn, ev in zip(ns, ell_values) if ev < nn), None)
if crossover:
    axes[1].axvline(
        x=crossover, color="green", linestyle=":", alpha=0.7, label=f"Crossover ~ n={crossover}"
    )
axes[1].set_xlabel("n (number of variables)")
axes[1].set_ylabel("Random values needed")
axes[1].set_title(r"$\ell$ vs n")
axes[1].set_xscale("log")
axes[1].set_yscale("log")
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Poking at Conjecture 3

**Conjecture 3** from the paper:

> $L_{1,2}(\text{Poly}_{n,d}) = O(d^2)$. That is, if $p : \mathbb{F}_2^n \to \mathbb{F}_2$ has degree $d$ and $f(x) = (-1)^{p(x)}$, then $\sum_{i<j} |\hat{f}(\{i,j\})| = O(d^2)$.

This is unproven. The known bound is $4 \cdot 2^{6d}$ (exponential). If the conjecture holds, it gives PRGs for AC0[$\oplus$].

We can't say anything about the conjecture from small experiments. But we can compute $L_{1,2}$ for many random instances and see what the numbers look like. Take this as an anecdote, not evidence.

In [ ]:
# Generate many random F2 polynomials at various degrees and measure L_{1,2}
n_conj = 12
degrees = [1, 2, 3, 4, 5]
n_samples_per_degree = 80
conj_rng = np.random.default_rng(seed=123)

results_by_degree = {d_val: [] for d_val in degrees}

for d_val in degrees:
    for _ in range(n_samples_per_degree):
        monomials = []
        for deg in range(1, d_val + 1):
            for combo in combinations(range(n_conj), deg):
                if conj_rng.random() < 0.3:  # include each monomial with prob 0.3
                    monomials.append(set(combo))
        if not monomials:
            monomials = [{0}]
        f = bf.f2_polynomial(n_conj, monomials)
        l12 = fourier_level_lp_norm(f, 2)
        results_by_degree[d_val].append(l12)

# Summary statistics
print(f"L_{{1,2}} for random degree-d F2 polynomials on n = {n_conj} variables")
print(f"({n_samples_per_degree} samples per degree)")
print()
print(f"{'d':<6}{'mean L_{1,2}':<16}{'max L_{1,2}':<16}{'known bound':<16}{'d^2':<8}")
print("-" * 62)
for d_val in degrees:
    vals = results_by_degree[d_val]
    known = 4 * 2 ** (6 * d_val)
    print(
        f"{d_val:<6}{np.mean(vals):<16.4f}{np.max(vals):<16.4f}{known:<16}{d_val**2:<8}"
    )

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: box plot of L_{1,2} by degree
box_data = [results_by_degree[d_val] for d_val in degrees]
bp = axes[0].boxplot(box_data, labels=[str(d_val) for d_val in degrees], patch_artist=True)
for patch in bp["boxes"]:
    patch.set_facecolor("steelblue")
    patch.set_alpha(0.6)
axes[0].set_xlabel("GF(2) degree d")
axes[0].set_ylabel("$L_{1,2}(f)$")
axes[0].set_title(f"$L_{{1,2}}$ of random degree-$d$ polynomials (n={n_conj})")
axes[0].grid(True, alpha=0.3, axis="y")

# Right: max L_{1,2} vs d, compared to d^2 and 4*2^{6d}
max_vals = [np.max(results_by_degree[d_val]) for d_val in degrees]
mean_vals = [np.mean(results_by_degree[d_val]) for d_val in degrees]

axes[1].plot(degrees, max_vals, "s-", color="steelblue", label="max $L_{1,2}$ (empirical)", linewidth=2)
axes[1].plot(degrees, mean_vals, "o-", color="orange", label="mean $L_{1,2}$ (empirical)", linewidth=2)
axes[1].plot(
    degrees,
    [d_val**2 for d_val in degrees],
    "k--",
    label="$d^2$ (Conjecture 3)",
    linewidth=1.5,
)
axes[1].set_xlabel("GF(2) degree d")
axes[1].set_ylabel("$L_{1,2}$")
axes[1].set_title("Empirical $L_{1,2}$ vs conjectured bound")
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The known bound 4 * 2^{6d} is not plotted -- it would be off the chart.")
print(f"For d=2 alone, the known bound is {4 * 2**12}.")

## Summary

We walked through the fractional PRG construction from CHLT'19:

| Step | What | Paper reference |
|------|------|-----------------|
| Fourier tails | Compute $L_{1,2}$ for the test class | Theorem 2 |
| Gaussian fooling | Small-variance, small-covariance Gaussians fool $\tilde{f}$ | Theorem 9 |
| Balanced code | Matrix $A$ maps $\ell$ Gaussians to $n$ with controlled covariance | Section 2, Step I |
| Discretization | Approximate each Gaussian with $k$ bits | Lemma 10-11 |
| Truncation | Clip to $[-1,1]^n$ | Claim 17 |

The full Boolean PRG adds a **polarizing random walk** (Theorem 7) to round $[-1,1]^n$ to $\{-1,+1\}^n$.

At $n = 12$, the construction uses more random bits than it saves. The seed length $\ell = O((t/\varepsilon)^{2+o(1)} \cdot \text{polylog}(n))$ only beats $n$ for large $n$ with moderate $t$. What matters at this scale is seeing each piece of the construction work.

On Conjecture 3: the empirical $L_{1,2}$ values we saw for random degree-$d$ polynomials were consistent with $O(d^2)$, and nowhere near the known bound of $4 \cdot 2^{6d}$. This is on $n = 12$ with random instances, so it says nothing about the conjecture in general. But the gap between the empirical numbers and the known bound is hard to ignore.

### Reference

Chattopadhyay, Hatami, Lovett, Tal. [Pseudorandom Generators from the Second Fourier Level and Applications to AC0 with Parity Gates](https://doi.org/10.4230/LIPIcs.ITCS.2019.22). *ITCS 2019*.